In [2]:
# =====================================================
# CELDA 1: Importación de librerías
# =====================================================
import pandas as pd   # Librería para manejo de datos (tablas, CSV, etc.)
import numpy as np    # Librería para operaciones numéricas
import matplotlib.pyplot as plt  # Librería para gráficos básicos
import seaborn as sns  # Librería para gráficos estadísticos
from sklearn.model_selection import train_test_split  # Para dividir dataset en train/test
from sklearn.preprocessing import StandardScaler      # Para escalar datos
from sklearn.linear_model import LogisticRegression   # Modelo de regresión logística
from sklearn.ensemble import RandomForestClassifier   # Modelo de Random Forest
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # Métricas

# La siguiente línea permite ver las gráficas dentro del Notebook
%matplotlib inline